# 04 UTA_Electric_Active Energy (kWh)

In [19]:
import pandas as pd

In [22]:
cd ..

c:\Users\Usuario\OneDrive\Documents\IE


## Definition of the constrain equation

This formula allows you to manage the UTA’s energy consumption by adjusting settings while considering operational and material constraints, ensuring optimal energy management without affecting the testing output quality.

**UTA_Electric_Active Energy = a⋅operational_presence + b⋅fabric_in_chamber + c⋅testing_schedule + d⋅workload + e⋅standby_power_down**

- **UTA_Electric_Active Energy** = Energy consumption of the Ultrasonic Testing Apparatus in a given period.

- **operational_presence** = Operational Presence: Indicates if the UTA is actively in operation - Given.

- **fabric_in_chamber** = Fabric in Chamber: Whether there is fabric loaded in the testing chamber, suggesting the UTA is about to be used - Given.

- **testing_schedule** = Testing Schedule: Reflects planned testing times which predict active use - Given.

- **workload** = Workload: Measures the queue of testing tasks assigned to the UTA - Given.

- **standby_power_down** = Standby Power Down: Managing power usage when the UTA is not actively testing - Controllable.

- **a,b,c,d,e** = Coefficients for each factor.

In [21]:
df = pd.read_csv('data/processed_data/industrial_sites_processed/2024-07-05T01-00-40_industrial_site2_processed.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/processed_data/industrial_sites_processed/2024-07-05T01-00-40_industrial_site2_processed.csv'

In [ ]:
df.columns

Index(['Time', '02 Production_Electric_Active Energy (kWh)',
       '02 Production_Electric_Active Power (kW)',
       '02 Production_Electric_Apparent Power (kVa)',
       '02 Production_Electric_Power Factor (real)',
       '03 Chiller Group_Electric_Active Energy (kWh)',
       '03 Chiller Group_Electric_Active Power (kW)',
       '03 Chiller Group_Electric_Apparent Power (kVa)',
       '03 Chiller Group_Electric_Power Factor (real)',
       '04 UTA_Electric_Active Energy (kWh)',
       '04 UTA_Electric_Active Power (kW)',
       '04 UTA_Electric_Apparent Power (kVa)',
       '04 UTA_Electric_Power Factor (real)',
       '05 Compressors_Electric_Active Energy (kWh)',
       '05 Compressors_Electric_Active Power (kW)',
       '05 Compressors_Electric_Apparent Power (kVa)',
       '05 Compressors_Electric_Power Factor (real)',
       '06 Offices_Electric_Active Energy (kWh)',
       '06 Offices_Electric_Active Power (kW)',
       '06 Offices_Electric_Apparent Power (kVa)',
       '06 

In [ ]:
UTA_target = (df['04 UTA_Electric_Active Energy (kWh)'])
UTA_target.to_csv('data/target_variables/target_UTA.csv', index=False)

## Other factors to include in the constrain equation

To effectively simulate data for the UTA (Ultrasonic Testing Apparatus) based on the equation, we first need to establish logical boundaries for each variable. These boundaries should reflect realistic operational scenarios and constraints. Here’s a suggested framework:

### 1. **operational_presence**
   - **Type:** Binary (0 or 1)
   - **Minimum:** 0 (UTA is not in operation)
   - **Maximum:** 1 (UTA is actively in operation)

### 2. **fabric_in_chamber**
   - **Type:** Binary (0 or 1)
   - **Minimum:** 0 (No fabric loaded in the chamber)
   - **Maximum:** 1 (Fabric is loaded, indicating readiness or impending use)

### 3. **testing_schedule**
   - **Type:** Continuous or Categorical (e.g., hours of operation)
   - **Minimum:** 0 (No scheduled tests)
   - **Maximum:** 24 (Reflects the maximum hours in a day that testing can be scheduled, if using hours as the measure)

### 4. **workload**
   - **Type:** Continuous
   - **Minimum:** 0 (No tests queued)
   - **Maximum:** 100 (Maximum number of tests that can be queued based on operational capacity)

### 5. **standby_power_down**
   - **Type:** Binary or Continuous (0 to 1 scale if continuous, reflecting degree of power reduction)
   - **Minimum:** 0 (Full power consumption, no power-saving measures in place)
   - **Maximum:** 1 (Fully powered down to the lowest safe operational level)

### Interaction Effects

**1. Operational Presence and Fabric in Chamber:**
   - When both `operational_presence` and `fabric_in_chamber` are active (both set to 1), the energy consumption could potentially be higher than the sum of their individual impacts due to the readiness of the UTA for immediate testing. This might require an additional coefficient in the model to account for this interaction if data shows a significant synergy effect.

**2. Workload and Testing Schedule:**
   - A higher `workload` could make the `testing_schedule` more packed, potentially increasing energy consumption as the UTA may operate continuously without breaks. Conversely, a sparse `testing_schedule` with a high `workload` might lead to inefficient operation as the apparatus starts and stops frequently.

**3. Standby Power Down and Operational Presence:**
   - The effectiveness of `standby_power_down` settings could be diminished if `operational_presence` is high, as the apparatus might rarely enter a low-power state. This interaction might suggest a need for dynamic power management strategies that adapt to operational rhythms.

**4. Fabric in Chamber and Standby Power Down:**
   - If fabric is loaded (`fabric_in_chamber` = 1) but there is a delay or cancellation in testing, the `standby_power_down` feature could be crucial in reducing idle energy consumption. This interaction points to the potential for smart scheduling or automatic delay handling to optimize energy use.

### Scenarios

**1. Peak Testing Periods:**
   - During peak operational times, all variables might interact to maximize energy consumption. Scenarios could include end-of-batch testing rushes or pre-maintenance test marathons, where workload is high, operational presence is continuous, and fabric loading is frequent.

**2. Maintenance or Downtime Days:**
   - On days scheduled for maintenance or when the UTA is not in use, `standby_power_down` could be maximized, `operational_presence` and `fabric_in_chamber` minimized, and `testing_schedule` cleared. Scenarios here help model the energy savings potential during non-operational periods.

**3. Unexpected Delays or Operational Hiccups:**
   - Scenarios involving unexpected machine downtime or delays in testing due to logistical issues could see high `fabric_in_chamber` with low `operational_presence`, affecting how `standby_power_down` strategies are implemented.

**4. Variable Load Days:**
   - Days when the testing demand varies significantly could be used to test the responsiveness of the energy consumption model to changes in `workload` and `testing_schedule`. These days can help fine-tune the interaction coefficients and power management strategies.

Here are some key statistics from the total energy consumption data:

In [ ]:
# Calculating statistical details for the total energy consumption data
UTA_stats = df['04 UTA_Electric_Active Energy (kWh)'].describe()
UTA_stats

count    10690.000000
mean        23.201022
std         15.912086
min          0.000000
25%          2.089500
50%         30.650000
75%         34.100000
max         55.275000
Name: 04 UTA_Electric_Active Energy (kWh), dtype: float64

## Synthesized dataset

In [ ]:
import pandas as pd
import numpy as np

# Number of samples is equal to the number of entries in the consumption data
n_samples = len(UTA_target)

# Initialize the random seed for reproducibility
np.random.seed(42)

# Define dynamic ranges based on energy consumption percentiles
max_energy = df['02 Production_Electric_Active Energy (kWh)'].max()
scale_factor = df['02 Production_Electric_Active Energy (kWh)'] / max_energy

# Generating data within the defined boundaries
data = {
    'power_transport_vehicles': np.random.uniform(0, 200, n_samples) * scale_factor,
    'production_schedule': np.round(np.random.uniform(0, 100, n_samples) * scale_factor),
    'maintenance_status': np.random.uniform(0, 100, n_samples),
    'volume_production_waste': np.random.uniform(0, 50, n_samples) * scale_factor,
    'number_of_workers': np.round(np.random.uniform(0, 500, n_samples) * scale_factor)
}

df_simulated = pd.DataFrame(data)

# Save the generated data to a new CSV
output_path = 'data/synthesized_data/production_synthesized.csv'
df_simulated.to_csv(output_path, index=False)

output_path